In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import BatchNorm1d

class BiLSTMModel(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, num_layers=3, dropout=0.5):
        super(BiLSTMModel, self).__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)#nn.Linear(input_dim, embedding_dim) # 
        
        # Bidirectional LSTM layers
        self.bilstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            bidirectional=True,
            batch_first=True,
            dropout=dropout,
        )
        # batch norm
        #prev_layer_size = hidden_dim * 2  # For bidirectional LSTM
        #self.batchnorm = BatchNorm1d(prev_layer_size)
        # Fully connected layers
        # self.fc1 = nn.Linear(hidden_dim * 2, 512)
        # self.fc2 = nn.Linear(512, 512)
        self.fc_out = nn.Linear(hidden_dim * 2, output_dim)

        # Dropout layer
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Embedding layer
        embedded = self.embedding(x)  # (batch_size, seq_len, embedding_dim)

        # BiLSTM layers
        lstm_out, _ = self.bilstm(embedded)  # (batch_size, seq_len, hidden_dim * 2)

        #batch norm
        #lstm_out = self.batchnorm(lstm_out)
        
        # TimeDistributed fully connected layers
        # output = self.fc1(lstm_out)  # (batch_size, seq_len, 512)
        # output = F.relu(output)
        # output = self.dropout(output)

        # output = self.fc2(output)  # (batch_size, seq_len, 512)
        # output = F.relu(output)
        # output = self.dropout(output)
        output = self.fc_out(lstm_out)  # (batch_size, seq_len, output_dim)
        return F.log_softmax(output, dim=-1)



In [4]:
import pandas as pd
train_path = '../dataset/PIZZA_train.json'
test_path = '../dataset/PIZZA_dev.json'
df = pd.read_json(train_path, lines=True)
dev = pd.read_json(test_path, lines=True)

In [49]:
df.describe()

,train.SRC,train.EXR,train.TOP,train.TOP-DECOUPLED
count,2456446,2456446,2456446,2456446
unique,2456446,694346,2456446,1425035
top,can i have a large bbq pulled pork,(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE PARTY_SIZ...,(ORDER can i have (PIZZAORDER (NUMBER a ) (SIZ...,(ORDER (PIZZAORDER (NUMBER three ) (NOT (TOPPI...
freq,1,1999,1,167


In [ ]:
unique_patterns = df['train.EXR'].value_counts()
unique_patterns.describe()
# Subset the DataFrame for rows with unique values
unique_related_dataset = df[df['train.EXR'].isin(unique_patterns.index[:694345])] # reduce to 5000 patterns

# Describe the resulting dataset
unique_related_dataset.describe()

,train.SRC,train.EXR,train.TOP,train.TOP-DECOUPLED
count,2456445,2456445,2456445,2456445
unique,2456445,694345,2456445,1425034
top,can i have a large bbq pulled pork,(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE PARTY_SIZ...,(ORDER can i have (PIZZAORDER (NUMBER a ) (SIZ...,(ORDER (PIZZAORDER (NUMBER three ) (NOT (TOPPI...
freq,1,1999,1,167


In [6]:
unique_related_dataset =unique_related_dataset.reset_index(drop=True)
unique_related_dataset

,train.SRC,train.EXR,train.TOP,train.TOP-DECOUPLED
0,can i have a large bbq pulled pork,(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE LARGE ) (...,(ORDER can i have (PIZZAORDER (NUMBER a ) (SIZ...,(ORDER (PIZZAORDER (NUMBER a ) (SIZE large ) (...
1,party size stuffed crust pie with american che...,(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE PARTY_SIZ...,(ORDER (PIZZAORDER (SIZE party size ) (STYLE s...,(ORDER (PIZZAORDER (SIZE party size ) (STYLE s...
2,can i have one personal sized artichoke,(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE PERSONAL_...,(ORDER can i have (PIZZAORDER (NUMBER one ) (S...,(ORDER (PIZZAORDER (NUMBER one ) (SIZE persona...
3,pie with banana pepper and peppperonis and ext...,(ORDER (PIZZAORDER (NUMBER 1 ) (TOPPING BANANA...,(ORDER (PIZZAORDER pie with (TOPPING banana pe...,(ORDER (PIZZAORDER (TOPPING banana pepper ) (T...
4,i want one regular pizza without any fried onions,(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE REGULARSI...,(ORDER i want (PIZZAORDER (NUMBER one ) (SIZE ...,(ORDER (PIZZAORDER (NUMBER one ) (SIZE regular...
...,...,...,...,...
720612,i'd like a party - size pizza with chicken pep...,(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE PARTY_SIZ...,(ORDER i'd like (PIZZAORDER (NUMBER a ) (SIZE ...,(ORDER (PIZZAORDER (NUMBER a ) (SIZE party - s...
720613,i'd like a party sized pizza with green olive ...,(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE PARTY_SIZ...,(ORDER i'd like (PIZZAORDER (NUMBER a ) (SIZE ...,(ORDER (PIZZAORDER (NUMBER a ) (SIZE party siz...
720614,i'd like a personal - sized pizza with green o...,(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE PERSONAL_...,(ORDER i'd like (PIZZAORDER (NUMBER a ) (SIZE ...,(ORDER (PIZZAORDER (NUMBER a ) (SIZE personal ...
720615,i'd like a party - size pizza with green olive...,(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE PARTY_SIZ...,(ORDER i'd like (PIZZAORDER (NUMBER a ) (SIZE ...,(ORDER (PIZZAORDER (NUMBER a ) (SIZE party - s...


In [7]:
X_train = unique_related_dataset['train.SRC']
y_train = unique_related_dataset['train.EXR']
X_test = dev['dev.SRC']
y_test = dev['dev.EXR']
print(len(unique_related_dataset))
print(X_train[476368])
print(y_train[476368])
print(dev['dev.SRC'][0])

720617
i'd like four party size pizzas no green olive
(ORDER (PIZZAORDER (NUMBER 4 ) (SIZE PARTY_SIZE ) (NOT (TOPPING GREEN_OLIVES ) ) ) )
i want to order two medium pizzas with sausage and black olives and two medium pizzas with pepperoni and extra cheese and three large pizzas with pepperoni and sausage


In [8]:
import sys
sys.path.append("..")
from utils.data_preprocessing import preprocess_text
from utils.feature_extraction import bag_of_words, tfidf_features, extract_embeddings
X_train = [" ".join(preprocess_text(text)) for text in X_train]
X_test = [" ".join(preprocess_text(text)) for text in X_test]

In [9]:
X_train

['large bbq pulled pork',
 'party size stuffed crust pie with american cheese with mushroom',
 'one personal sized artichoke',
 'pie with banana pepper peppperonis low fat cheese',
 'want one regular pizza without any fried onion',
 'want stuffed crust pizza with american cheese little bit peperonni',
 'one party sized high rise dough pizza with american cheese lot peperonni',
 'one pie with banana pepper peperonis yellow pepper',
 'like one party sized pie with american cheese with pesto sauce',
 'one high rise dough pie with american cheese lot meatball',
 'party sized pie without any bean',
 'party sized big meat pizza with american cheese mushroom',
 'like party sized high rise dough pie with lot banana pepper pecorino cheese',
 'one party size high rise dough pie with green olive',
 'like party sized stuffed crust pie with banana pepper pesto',
 'like lunch sized pie without alfredo chicken',
 'want one personal sized pizza without any bean',
 'one party size mediterranean pie',
 

In [10]:
max_str_1 = len(max(X_train, key=len))
max_str_2 = len(y_train[y_train.str.len().idxmax()])
max_str_1, max_str_2

(113, 265)

In [11]:
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences

def tokenize_output(output):
    """
    Tokenizes the structured output into meaningful tokens.
    Example:
        Input: "(ORDER (PIZZAORDER (NUMBER a ) (SIZE large ) (TOPPING bbq pulled pork ) ) )"
        Output: ["(ORDER", "(PIZZAORDER", "(NUMBER", "a", "(SIZE", "large", "(TOPPING", "bbq", "pulled", "pork", ")", ")", ")", ")"]
    """
    tokens = re.findall(r"\(|\)|\w+|[^\s()]+", output)
    return tokens

def build_vocab(outputs):
    """
    Builds a vocabulary from tokenized outputs.
    """
    vocab = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2}  # Special tokens
    i = 3
    for output in outputs:
        tokens = tokenize_output(output)
        for token in tokens:
            if token not in vocab:
                vocab[token] = i
                i += 1
    return vocab
def encode_outputs(outputs, vocab):
    """
    Encodes tokenized outputs into sequences of integers.
    """
    encoded = []
    for output in outputs:
        tokens = tokenize_output(output)
        sequence = [vocab["<SOS>"]] + [vocab[token] for token in tokens if token in vocab] + [vocab["<EOS>"]]
        encoded.append(sequence)
    return encoded

def pad_sequences_to_fixed_length(sequences, max_len):
    """
    Pads sequences to a fixed length.
    """
    return pad_sequences(sequences, maxlen=max_len, padding="post", value=0)

def decode_sequence(sequence, vocab):
    """
    Decodes a sequence of integers back into the structured output string.
    """
    inv_vocab = {v: k for k, v in vocab.items()}  # Reverse the vocabulary
    tokens = [inv_vocab[idx] for idx in sequence if idx in inv_vocab and idx not in {vocab["<SOS>"], vocab["<EOS>"], vocab["<PAD>"]}]

    output = " ".join(tokens)
    output = output.replace(" ( ", " (").replace("( ", "(") #.replace(" )", ")")
    return output

def decode_sequence_2(sequence, vocab):
    """
    Decodes a sequence of integers back into the structured output string.
    """
    inv_vocab = {v: k for k, v in vocab.items()}  # Reverse the vocabulary
    # sequence = sequence.cpu().tolist()  # Convert tensor to a list of integers
    tokens = [inv_vocab.get(idx, "") for idx in sequence if idx > 0]  # Ignore unknown and put empty char
    return "".join(tokens)



In [12]:
# import gensim.downloader as api
# glove_vectors = api.load("glove-wiki-gigaword-100")  # 100-dimension GloVe
# glove_vectors

In [13]:


def prepare_data(
    X_train, y_train, X_test, y_test, feature_type="bow", glove_vectors=None, max_len_1=20, max_len_2 = 20
):

    vectorizer = None

    # # Feature Extraction for X_train and X_test
    # if feature_type == "bow":
    #     X_train_processed, vectorizer = bag_of_words(X_train)
    #     X_train_processed = X_train_processed.toarray()
    #     X_test_processed = vectorizer.transform(X_test).toarray()
    # elif feature_type == "tfidf":
    #     X_train_processed, vectorizer = tfidf_features(X_train)
    #     X_train_processed = X_train_processed.toarray()
    #     X_test_processed = vectorizer.transform(X_test).toarray()
    # elif feature_type == "embeddings":
    #     if not glove_vectors:
    #         raise ValueError("GloVe vectors must be provided for embeddings.")
    #     X_train_tokenized = [sentence.split() for sentence in X_train]
    #     X_test_tokenized = [sentence.split() for sentence in X_test]
    #     X_train_processed = extract_embeddings(X_train_tokenized)
    #     X_test_processed = extract_embeddings(X_test_tokenized)
    # else:
    #     raise ValueError("Invalid feature type. Choose 'bow', 'tfidf', or 'embeddings'.")
    
    X_vocab = build_vocab(X_train)  # Build vocabulary from training outputs
    X_train_encoded = encode_outputs(X_train, X_vocab)  # Encode training outputs
    X_test_encoded = encode_outputs(X_test, X_vocab)  # Encode testing outputs
    X_train_processed = pad_sequences_to_fixed_length(X_train_encoded, max_len_1)
    X_test_processed = pad_sequences_to_fixed_length(X_test_encoded, max_len_1)

    vocab = build_vocab(y_train)  # Build vocabulary from training outputs
    y_train_encoded = encode_outputs(y_train, vocab)  # Encode training outputs
    y_test_encoded = encode_outputs(y_test, vocab)  # Encode testing outputs
    y_train_processed = pad_sequences_to_fixed_length(y_train_encoded, max_len_2)
    y_test_processed = pad_sequences_to_fixed_length(y_test_encoded, max_len_2)


    return (
        X_train_processed,
        X_test_processed,
        y_train_processed,
        y_test_processed,
        X_vocab,
        vocab,  # Return vocabulary for decoding
    )


In [14]:
X_train_processed, X_test_processed, y_train_processed, y_test_processed, X_vocab, vocab = prepare_data( X_train, y_train, X_test, y_test, feature_type="embeddings", max_len_1=300, max_len_2=300)

In [15]:
X_vocab

{'<PAD>': 0,
 '<SOS>': 1,
 '<EOS>': 2,
 'large': 3,
 'bbq': 4,
 'pulled': 5,
 'pork': 6,
 'party': 7,
 'size': 8,
 'stuffed': 9,
 'crust': 10,
 'pie': 11,
 'with': 12,
 'american': 13,
 'cheese': 14,
 'mushroom': 15,
 'one': 16,
 'personal': 17,
 'sized': 18,
 'artichoke': 19,
 'banana': 20,
 'pepper': 21,
 'peppperonis': 22,
 'low': 23,
 'fat': 24,
 'want': 25,
 'regular': 26,
 'pizza': 27,
 'without': 28,
 'any': 29,
 'fried': 30,
 'onion': 31,
 'little': 32,
 'bit': 33,
 'peperonni': 34,
 'high': 35,
 'rise': 36,
 'dough': 37,
 'lot': 38,
 'peperonis': 39,
 'yellow': 40,
 'like': 41,
 'pesto': 42,
 'sauce': 43,
 'meatball': 44,
 'bean': 45,
 'big': 46,
 'meat': 47,
 'pecorino': 48,
 'green': 49,
 'olive': 50,
 'lunch': 51,
 'alfredo': 52,
 'chicken': 53,
 'mediterranean': 54,
 'caramelized': 55,
 'red': 56,
 'no': 57,
 'apple': 58,
 'wood': 59,
 'bacon': 60,
 'combination': 61,
 'mozzarella': 62,
 'chorrizo': 63,
 'peperroni': 64,
 'basil': 65,
 'feta': 66,
 'medium': 67,
 'need': 6

In [16]:
vocab

{'<PAD>': 0,
 '<SOS>': 1,
 '<EOS>': 2,
 '(': 3,
 'ORDER': 4,
 'PIZZAORDER': 5,
 'NUMBER': 6,
 '1': 7,
 ')': 8,
 'SIZE': 9,
 'LARGE': 10,
 'TOPPING': 11,
 'BBQ_PULLED_PORK': 12,
 'PARTY_SIZE': 13,
 'STYLE': 14,
 'STUFFED_CRUST': 15,
 'AMERICAN_CHEESE': 16,
 'MUSHROOMS': 17,
 'PERSONAL_SIZE': 18,
 'ARTICHOKES': 19,
 'BANANA_PEPPERS': 20,
 'PEPPERONI': 21,
 'COMPLEX_TOPPING': 22,
 'QUANTITY': 23,
 'EXTRA': 24,
 'LOW_FAT_CHEESE': 25,
 'REGULARSIZE': 26,
 'NOT': 27,
 'FRIED_ONIONS': 28,
 'LIGHT': 29,
 'THICK_CRUST': 30,
 'YELLOW_PEPPERS': 31,
 'PESTO': 32,
 'MEATBALLS': 33,
 'BEANS': 34,
 'MEAT_LOVER': 35,
 'PECORINO_CHEESE': 36,
 'GREEN_OLIVES': 37,
 'LUNCH_SIZE': 38,
 'ALFREDO_CHICKEN': 39,
 'MEDITERRANEAN': 40,
 'CARAMELIZED_ONIONS': 41,
 'BACON': 42,
 'COMBINATION': 43,
 'MOZZARELLA_CHEESE': 44,
 'CHORIZO': 45,
 'CHEESE': 46,
 'BASIL': 47,
 'FETA_CHEESE': 48,
 'MEDIUM': 49,
 'SAUCE': 50,
 'CUMIN': 51,
 'SMALL': 52,
 'ANCHOVIES': 53,
 'GREEN_PEPPERS': 54,
 'CHEESEBURGER': 55,
 'BUFFALO_C

In [17]:
X_train_processed.shape, y_train_processed.shape

((720617, 300), (720617, 300))

In [18]:
X_train_processed[1]

array([ 1,  7,  8,  9, 10, 11, 12, 13, 14, 12, 15,  2,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0

In [19]:
y_train_processed[1]

array([ 1,  3,  4,  3,  5,  3,  6,  7,  8,  3,  9, 13,  8,  3, 14, 15,  8,
        3, 11, 16,  8,  3, 11, 17,  8,  8,  8,  2,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0

In [20]:

ou= decode_sequence(y_train_processed[1],vocab)
ou

'(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE PARTY_SIZE ) (STYLE STUFFED_CRUST ) (TOPPING AMERICAN_CHEESE ) (TOPPING MUSHROOMS ) ) )'

In [21]:
y_train[1]

'(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE PARTY_SIZE ) (STYLE STUFFED_CRUST ) (TOPPING AMERICAN_CHEESE ) (TOPPING MUSHROOMS ) ) )'

In [22]:
ou == y_train[1]

True

### Ensure Decoding works right

In [48]:
total_sequences = 0
correct_sequences = 0

for src,tgt in zip(y_test_processed, y_test):
    #print(src)

    predicted_sequences = decode_sequence(src, vocab) 
    #print(predicted_sequences)
    # print(predicted_sequences)
    # print(target_sequences)
    # Calculate sequence accuracy
    if predicted_sequences == tgt:
        # print(pred)
        # print(tgt)
        correct_sequences += 1
    total_sequences += 1
print(f"Correct {correct_sequences}, Total {total_sequences}")
sequence_accuracy = correct_sequences / total_sequences if total_sequences > 0 else 0
sequence_accuracy * 100

Correct 306, Total 348


87.93103448275862

In [30]:
len(vocab)

158

In [31]:
input_dim = X_train_processed.shape[1]
input_dim

300

In [32]:
output_dim = y_train_processed.shape[1] 
output_dim

300

In [33]:
input_dim = X_train_processed.shape[1]  # Vocabulary size
embedding_dim = 300  # Dimension of embedding vectors    vocabs  260,158       max seq lens  (99, 265)
hidden_dim = 256  # Hidden state size for LSTM          first was 256 for batch norm 150
output_dim = y_train_processed.shape[1]  # Number of output classes
num_layers = 3  # Number of BiLSTM layers
dropout = 0.5  # Dropout probability

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTMModel(input_dim, embedding_dim, hidden_dim, output_dim, num_layers, dropout).to(device)


In [34]:
import torch
from torch.utils.data import Dataset, DataLoader

class SequenceDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = torch.tensor(inputs, dtype=torch.long) 
        self.targets = torch.tensor(targets, dtype=torch.long) 

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return {
            "src_input_ids": self.inputs[idx],
            "tgt_input_ids": self.targets[idx],
        }

train_dataset = SequenceDataset(X_train_processed, y_train_processed)
test_dataset = SequenceDataset(X_test_processed, y_test_processed)

batch_size = 64  # Adjust based GPU ;-;  memory
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [35]:
train_dataset.__getitem__(3)

{'src_input_ids': tensor([ 1, 11, 12, 20, 21, 22, 23, 24, 14,  2,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0

In [36]:
def evaluate_model_with_accuracy(model, dataloader, criterion, device):
    model.eval()
    epoch_loss = 0
    total_tokens = 0
    correct_tokens = 0

    with torch.no_grad():
        for batch in dataloader:
            src = batch["src_input_ids"].to(device)
            tgt = batch["tgt_input_ids"].to(device)

            output = model(src)
            output_dim = output.shape[-1]
            output = output[:, 1:].reshape(-1, output_dim)  # Reshape for token comparison
            tgt = tgt[:, 1:].reshape(-1)

            # Calculate loss
            loss = criterion(output, tgt)
            epoch_loss += loss.item()

            # Calculate accuracy
            predictions = output.argmax(dim=1)  # Get the index of the max log-probability
            correct_tokens += (predictions == tgt).sum().item()
            total_tokens += tgt.size(0)

    accuracy = correct_tokens / total_tokens if total_tokens > 0 else 0
    return epoch_loss / len(dataloader), accuracy

In [37]:
import torch.optim as optim
from tqdm import tqdm

criterion = nn.CrossEntropyLoss()  # Use for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(3):  # Number of epochs
    model.train()
    epoch_loss = 0
    total_batches = len(train_dataloader)
    progress_bar = tqdm(train_dataloader,desc="Training Progress", unit="batch", leave=True)
    for  batch_idx, batch in enumerate(progress_bar): # Assuming a DataLoader is used for batches
        src = batch["src_input_ids"].to(device)  # Input tokens
        tgt = batch["tgt_input_ids"].to(device)  # Target tokens

        optimizer.zero_grad()
        output = model(src)  # Forward pass


        # Flatten the model output
        output = output.view(-1, output_dim) 
        tgt = tgt.view(-1)

        loss = criterion(output, tgt)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        avg_loss = epoch_loss / (batch_idx + 1)
        progress_bar.set_description(f"Training Progress: Batch {batch_idx + 1}/{total_batches}, Avg Loss: {avg_loss:.4f}")
    val_loss, accuracy = evaluate_model_with_accuracy(model, test_dataloader, criterion, device)
    print(f"Epoch {epoch + 1}, Loss: {epoch_loss / len(train_dataloader):.4f}, Val Loss: {val_loss / len(test_dataloader):.4f}, Accuracy: {accuracy * 100:.4f}%")




Training Progress: Batch 11260/11260, Avg Loss: 0.0211: 100%|██████████| 11260/11260 [32:45<00:00,  5.73batch/s]


Epoch 1, Loss: 0.0211, Val Loss: 0.1699, Accuracy: 93.0179%


Training Progress: Batch 11260/11260, Avg Loss: 0.0048: 100%|██████████| 11260/11260 [36:13<00:00,  5.18batch/s]  


Epoch 2, Loss: 0.0048, Val Loss: 0.1955, Accuracy: 93.0304%


Training Progress: Batch 172/11260, Avg Loss: 0.0041:   2%|▏         | 172/11260 [00:29<32:09,  5.75batch/s]


KeyboardInterrupt: 

### Saving the model

In [38]:
def save_model(model, path):
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

def load_model(model, path):
    model.load_state_dict(torch.load(path))
    print(f"Model loaded from {path}")
    return model

In [39]:
save_model(model, "../weights/Bilstm.pt")

Model saved to ../weights/Bilstm.pt


### Testing Real Output sequence

In [40]:
model = BiLSTMModel(input_dim, embedding_dim, hidden_dim, output_dim, num_layers, dropout).to(device)
model = load_model(model,"../weights/Bilstm.pt")
with torch.no_grad():
    output = model(test_dataset.__getitem__(1)["src_input_ids"].to(device))
    predictions = output.argmax(dim=1)
predictions

Model loaded from ../weights/Bilstm.pt


tensor([ 1,  3,  4,  3,  5,  3,  6,  7,  8,  3,  9, 49,  8,  3, 11, 77,  8,  3,
        11, 31,  8,  8,  8,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0, 

In [42]:
tgt= test_dataset.__getitem__(1)["tgt_input_ids"].to(device)
tgt

tensor([  1,   3,   4,   3,   5,   3,   6, 129,   8,   3,   9,  49,   8,   3,
         11,  77,   8,   3,  11, 104,   8,   8,   8,   2,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  

In [43]:

sequence = predictions.cpu().tolist()
pred_sequence= decode_sequence(sequence,vocab)
pred_sequence

'(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE MEDIUM ) (TOPPING HAM ) (TOPPING YELLOW_PEPPERS ) ) )'

In [44]:
sequence = tgt.cpu().tolist()
tgt_sequence= decode_sequence(sequence,vocab)
tgt_sequence

'(ORDER (PIZZAORDER (NUMBER 5 ) (SIZE MEDIUM ) (TOPPING HAM ) (TOPPING TOMATOES ) ) )'

In [45]:
pred_sequence == tgt_sequence

False

In [46]:
def evaluate_model_with_sequence_accuracy(model, dataloader, device):
    model.eval()
    total_sequences = 0
    correct_sequences = 0

    with torch.no_grad():
        for batch in dataloader:
            src = batch["src_input_ids"].to(device)
            tgt = batch["tgt_input_ids"].to(device)

            output = model(src)
            output = output.argmax(dim=-1)

            # Decode sequences for comparison

            predicted_sequences = [decode_sequence(seq.cpu().tolist(), vocab) for seq in output]
            target_sequences = [decode_sequence(seq.cpu().tolist(), vocab) for seq in tgt]
            # print(predicted_sequences)
            # print(target_sequences)
            # Calculate sequence accuracy
            for pred, tgt in zip(predicted_sequences, target_sequences):
                if pred == tgt:
                    print(pred)
                    print(tgt)
                    correct_sequences += 1
                total_sequences += 1
    print(f"Correct {correct_sequences}, Total {total_sequences}")
    sequence_accuracy = correct_sequences / total_sequences if total_sequences > 0 else 0
    return sequence_accuracy * 100

In [47]:
evaluate_model_with_sequence_accuracy(model, test_dataloader, device)

(ORDER (PIZZAORDER (NUMBER 1 ) (TOPPING FETA_CHEESE ) ) )
(ORDER (PIZZAORDER (NUMBER 1 ) (TOPPING FETA_CHEESE ) ) )
(ORDER (PIZZAORDER (NUMBER 1 ) ) )
(ORDER (PIZZAORDER (NUMBER 1 ) ) )
(ORDER (PIZZAORDER (NUMBER 3 ) (SIZE LARGE ) (TOPPING MUSHROOMS ) ) )
(ORDER (PIZZAORDER (NUMBER 3 ) (SIZE LARGE ) (TOPPING MUSHROOMS ) ) )
Correct 3, Total 348


0.8620689655172413